# Cancer Data Experimental ML Analysis v3

In [10]:
#import useful libraries
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from matplotlib.pyplot import figure
import sys
#import umap.umap_ as umap
import random
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import make_pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.decomposition import KernelPCA
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import KFold

C:\Users\richi_afzmjq6\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [ ]:
import sys
!{sys.executable} -m pip install umap-learn

In [ ]:
import umap.umap_ as umap

In [3]:
#import data (beware, it takes around 10 minutes)
df = pd.read_csv('../clinical_TumorCompendium_v11_PolyA_2020-04-09.tsv', sep='\t', header=0)
df2 = pd.read_csv('../TumorCompendium_v11_PolyA_hugo_log2tpm_58581genes_2020-04-09.tsv', sep='\t', header=0)

In [ ]:
def experiments_BACs(X_train, X_test, y_train, y_test, k):
    
    #CST goes here
    
    
    pca = PCA(n_components = k)
    pca.fit(X_train)
    X_train_trans = pca.transform(X_train)
    X_test_trans = pca.transform(X_test)
    pca_bacs = experiment_BACs_helper(X_train_trans, X_test_trans, y_train, y_test, k)
    
    selectKBest = SelectKBest(score_func=mutual_info_classif, k=4)
    selectKBest.fit(X_train, y_train)
    X_train_trans = selectKBest.transform(X_train)
    X_test_trans = selectKBest.transform(X_test)
    mi_bacs = experiment_BACs_helper(X_train_trans, X_test_trans, y_train, y_test, k)
    
    mapper = umap.UMAP(n_neighbors=(k*4)).fit(X_train, y_train)
    X_train_trans = mapper.transform(X_train)
    X_test_trans = mapper.transform(X_test)
    umap_bacs = experiment_BACs_helper(X_train_trans, X_test_trans, y_train, y_test, k)
    
    kpca = KernelPCA(n_components = k, kernel = 'poly')
    kpca.fit(X_train)
    X_train_trans = kpca.transform(X_train)
    X_test_trans = kpca.transform(X_test)
    kpca_bacs = experiment_BACs_helper(X_train_trans, X_test_trans, y_train, y_test, k)
    
    return  , pca_bacs, mi_bacs, umap_bacs, kpca_bacs
    

In [7]:
def experiment_BACs_helper(X_train_trans, X_test_trans, y_train, y_test, k):
    
    temp_BACs = np.zeros(6)
    
    neighbors = KNeighborsClassifier(n_neighbors=6)
    neighbors.fit(X_train_trans, y_train)
    temp_BACs[0] = balanced_accuracy_score(y_test, neighbors.predict(X_test_trans))
    
    svm = SVC(kernel='linear', C=1)
    svm.fit(X_train_trans, y_train)
    temp_BACs[1] = balanced_accuracy_score(y_test, svm.predict(X_test_trans))
    
    gnb = GaussianNB()
    gnb.fit(X_train_trans, y_train)
    temp_BACs[2] = balanced_accuracy_score(y_test, gnb.predict(X_test_trans))
    
    rf = RandomForestClassifier(random_state=137)
    rf.fit(X_train_trans, y_train)
    temp_BACs[3] = balanced_accuracy_score(y_test, rf.predict(X_test_trans))
    
    sgd = make_pipeline(StandardScaler(), SGDClassifier(max_iter=1000, tol=1e-3))
    sgd.fit(X_train_trans, y_train)
    temp_BACs[4] = balanced_accuracy_score(y_test, sgd.predict(X_test_trans))
    
    lda = LinearDiscriminantAnalysis()
    lda.fit(X_train_trans, y_train)
    temp_BACs[5] = balanced_accuracy_score(y_test, lda.predict(X_test_trans))
    
    return temp_BACs
    

In [8]:
#checks out bc 50 total iterations from for loops and each iteration conducts 30 ML experiments
def runexperiments_full_6(g1_samples, g2_samples, g3_samples, g4_samples, g5_samples, g6_samples):
    
    x = pd.concat([g1_samples.T, g2_samples.T, g3_samples.T, g4_samples.T, g5_samples.T, g6_samples.T]) #add together all postiive and negative labels 
    x = StandardScaler().fit_transform(x) #standardize values in x (z-score standardization)

    target_g1 = pd.DataFrame(np.zeros((len(g1_samples.T), 1))) #get number of target group 1
    target_g2 = pd.DataFrame(np.ones((len(g2_samples.T), 1))) #get number of target group 2
    target_g3 = pd.DataFrame(2*np.ones((len(g3_samples.T), 1))) #get number of target group 3
    target_g4 = pd.DataFrame(3*np.ones((len(g4_samples.T), 1))) #get number of target group 4
    target_g5 = pd.DataFrame(4*np.ones((len(g5_samples.T), 1))) #get number of target group 5
    target_g6 = pd.DataFrame(5*np.ones((len(g6_samples.T), 1))) #get number of target group 6

    target = pd.concat([target_g1, target_g2, target_g3, target_g4, target_g5, target_g6]) #concatenate target zeros ones twos etc together
    target = target.reset_index(drop=True)
    
    splits_rats = np.array([.1, .15, .2, .25, .3])

    #BAC_sums_cst = np.zeros(6)
    BAC_sums_pca = np.zeros(6)
    BAC_sums_mi = np.zeros(6)
    BAC_sums_umap = np.zeros(6)
    BAC_sums_kpca = np.zeros(6)
    
    split_num = 10
    
    for i in splits_rats:
        #split the data into test and train
        X_train, X_test, y_train, y_test = train_test_split(x, target.to_numpy(), test_size=i, random_state=42)
        y_train = y_train.ravel()
        y_test = y_test.ravel()

        
        kf = KFold(n_splits=split_num, shuffle=True, random_state=42)

        for trainIn, valIn in kf.split(X_train):
            
            x_train_nonval = X_train[trainIn]
            y_train_nonval = y_train[trainIn]

            x_val = X_train[valIn]
            y_val = y_train[valIn]
            
            t2, t3, t4, t5 = experiments_BACs(x_train_nonval, X_test, y_train_nonval, y_test, 6)
            #BAC_sums_cst += t1
            BAC_sums_pca += t2
            BAC_sums_mi += t3
            BAC_sums_umap += t4
            BAC_sums_kpca += t5
    
    numIter = splits_rats.shape[0]*split_num
    #BAC_sums_cst /= numIter
    BAC_sums_pca /= numIter
    BAC_sums_mi /= numIter
    BAC_sums_umap /= numIter
    BAC_sums_kpca /= numIter    
    
    return BAC_sums_pca, BAC_sums_mi, BAC_sums_umap, BAC_sums_kpca
    

In [ ]:
#checks out bc 50 total iterations from for loops and each iteration conducts 30 ML experiments
def runexperiments_full_4(g1_samples, g2_samples, g3_samples, g4_samples):
    
    x = pd.concat([g1_samples.T, g2_samples.T, g3_samples.T, g4_samples.T]) #add together all postiive and negative labels 
    x = StandardScaler().fit_transform(x) #standardize values in x (z-score standardization)

    target_g1 = pd.DataFrame(np.zeros((len(g1_samples.T), 1))) #get number of target group 1
    target_g2 = pd.DataFrame(np.ones((len(g2_samples.T), 1))) #get number of target group 2
    target_g3 = pd.DataFrame(2*np.ones((len(g3_samples.T), 1))) #get number of target group 3
    target_g4 = pd.DataFrame(3*np.ones((len(g4_samples.T), 1))) #get number of target group 4

    target = pd.concat([target_g1, target_g2, target_g3, target_g4]) #concatenate target zeros ones twos etc together
    target = target.reset_index(drop=True)
    
    splits_rats = np.array([.1, .15, .2, .25, .3])

    #BAC_sums_cst = np.zeros(6)
    BAC_sums_pca = np.zeros(6)
    BAC_sums_mi = np.zeros(6)
    BAC_sums_umap = np.zeros(6)
    BAC_sums_kpca = np.zeros(6)
    
    split_num = 10
    
    for i in splits_rats:
        #split the data into test and train
        X_train, X_test, y_train, y_test = train_test_split(x, target.to_numpy(), test_size=i, random_state=42)
        y_train = y_train.ravel()
        y_test = y_test.ravel()

        
        kf = KFold(n_splits=split_num, shuffle=True, random_state=42)

        for trainIn, valIn in kf.split(X_train):
            
            x_train_nonval = X_train[trainIn]
            y_train_nonval = y_train[trainIn]

            x_val = X_train[valIn]
            y_val = y_train[valIn]
            
            t2, t3, t4, t5 = experiments_BACs(x_train_nonval, X_test, y_train_nonval, y_test, 4)
            #BAC_sums_cst += t1
            BAC_sums_pca += t2
            BAC_sums_mi += t3
            BAC_sums_umap += t4
            BAC_sums_kpca += t5
    
    numIter = splits_rats.shape[0]*split_num
    #BAC_sums_cst /= numIter
    BAC_sums_pca /= numIter
    BAC_sums_mi /= numIter
    BAC_sums_umap /= numIter
    BAC_sums_kpca /= numIter    
    
    return BAC_sums_pca, BAC_sums_mi, BAC_sums_umap, BAC_sums_kpca
    

In [ ]:
#checks out bc 50 total iterations from for loops and each iteration conducts 30 ML experiments
def runexperiments_full_2(g1_samples, g2_samples):
    
    x = pd.concat([g1_samples.T, g2_samples.T]) #add together all postiive and negative labels 
    x = StandardScaler().fit_transform(x) #standardize values in x (z-score standardization)

    target_g1 = pd.DataFrame(np.zeros((len(g1_samples.T), 1))) #get number of target group 1
    target_g2 = pd.DataFrame(np.ones((len(g2_samples.T), 1))) #get number of target group 2

    target = pd.concat([target_g1, target_g2]) #concatenate target zeros ones twos etc together
    target = target.reset_index(drop=True)
    
    splits_rats = np.array([.1, .15, .2, .25, .3])

    #BAC_sums_cst = np.zeros(6)
    BAC_sums_pca = np.zeros(6)
    BAC_sums_mi = np.zeros(6)
    BAC_sums_umap = np.zeros(6)
    BAC_sums_kpca = np.zeros(6)
    
    split_num = 10
    
    for i in splits_rats:
        #split the data into test and train
        X_train, X_test, y_train, y_test = train_test_split(x, target.to_numpy(), test_size=i, random_state=42)
        y_train = y_train.ravel()
        y_test = y_test.ravel()

        
        kf = KFold(n_splits=split_num, shuffle=True, random_state=42)

        for trainIn, valIn in kf.split(X_train):
            
            x_train_nonval = X_train[trainIn]
            y_train_nonval = y_train[trainIn]

            x_val = X_train[valIn]
            y_val = y_train[valIn]
            
            t2, t3, t4, t5 = experiments_BACs(x_train_nonval, X_test, y_train_nonval, y_test, 2)
            #BAC_sums_cst += t1
            BAC_sums_pca += t2
            BAC_sums_mi += t3
            BAC_sums_umap += t4
            BAC_sums_kpca += t5
    
    numIter = splits_rats.shape[0]*split_num
    #BAC_sums_cst /= numIter
    BAC_sums_pca /= numIter
    BAC_sums_mi /= numIter
    BAC_sums_umap /= numIter
    BAC_sums_kpca /= numIter    
    
    return BAC_sums_pca, BAC_sums_mi, BAC_sums_umap, BAC_sums_kpca
    

In [4]:
#MI experiment
from sklearn.feature_selection import mutual_info_classif

samples7 = df[df['disease']=='lung adenocarcinoma']['th_sampleid'] 
g7_samples = df2.loc[:,list(samples7)]

samples8 = df[df['disease']=='stomach adenocarcinoma']['th_sampleid'] 
g8_samples = df2.loc[:,list(samples8)]

samples9 = df[df['disease']=='prostate adenocarcinoma']['th_sampleid'] 
g9_samples = df2.loc[:,list(samples9)]

samples10 = df[df['disease']=='colon adenocarcinoma']['th_sampleid'] 
g10_samples = df2.loc[:,list(samples10)]

x = pd.concat([g7_samples.T, g8_samples.T, g9_samples.T, g10_samples.T]) #add together all postiive and negative labels 
x = StandardScaler().fit_transform(x) #standardize values in x (z-score standardization)

target_g1 = pd.DataFrame(np.zeros((len(g7_samples.T), 1))) #get number of target group 1
target_g2 = pd.DataFrame(np.ones((len(g8_samples.T), 1))) #get number of target group 2
target_g3 = pd.DataFrame(2*np.ones((len(g9_samples.T), 1))) #get number of target group 3
target_g4 = pd.DataFrame(3*np.ones((len(g10_samples.T), 1))) #get number of target group 4

target = pd.concat([target_g1, target_g2, target_g3, target_g4]) #concatenate target zeros ones twos etc together
target = target.reset_index(drop=True)

#split the data into test and train
X_train, X_test, y_train, y_test = train_test_split(x, target.to_numpy(), test_size=0.3, random_state=42)
y_train = y_train.ravel()
y_test = y_test.ravel()

In [5]:
#edit this following code to fit an experiment
selectKBest = SelectKBest(score_func=mutual_info_classif, k=4)
selectKBest.fit(X_train, y_train)
X_train_trans = selectKBest.transform(X_train)
X_test_trans = selectKBest.transform(X_test)

In [6]:
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train_trans, y_train)
print(balanced_accuracy_score(y_test, neigh.predict(X_test_trans)))

0.8232731410027149


In [ ]:
#makes sure umap works (in HiPerGator - it does)
mapper = umap.UMAP(n_neighbors=(4*4)).fit(X_train, y_train)
X_train_trans = mapper.transform(X_train)
X_test_trans = mapper.transform(X_test)

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train_trans, y_train)
print(balanced_accuracy_score(y_test, neigh.predict(X_test_trans)))